In [3]:
import pandas as pd

# اقرأ كل ملف JSON
df_dos = pd.read_json(r"c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\400DOS.json" ,lines=True)
df_dos["label"] = "DOS"

df_meter = pd.read_json(r"c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\400METAR.json" ,lines=True)
df_meter["label"] = "METER"
df_port = pd.read_json(r"c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\120RPORT.json" ,lines=True)
df_port["label"] = "PORT SCANNING"

df_normal = pd.read_json(r"c:\Users\MARIAM\Downloads\project data\connections.json" ,lines=True)
df_normal["label"] = "NORMAL"

# دمجهم كلهم
df = pd.concat([df_dos, df_meter, df_port, df_normal], ignore_index=True)


In [4]:
df = df.drop(columns=["source_ip", "destination_ip", "protocol","is_well_known_port","connection_id"])


In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])  # مثلاً: NORMAL=3, DOS=0, ...


In [6]:
X = df.drop(columns=['label'])
y = df['label']


In [7]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X, y)


In [8]:
from collections import Counter

print("قبل SMOTE:", Counter(y))
print("بعد SMOTE:", Counter(y_resampled))


قبل SMOTE: Counter({0: 403, 1: 313, 2: 308, 3: 120})
بعد SMOTE: Counter({0: 403, 1: 403, 3: 403, 2: 403})


In [15]:
# فك التشفير
labels_resampled = le.inverse_transform(y_resampled)
# تحويلها لداتا فريم
df_resampled = pd.DataFrame(X_resampled)
df_resampled['label'] = labels_resampled
df_resampled = df_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

# حفظ كـ CSV
df_resampled.to_csv(r"c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\balanced_logs_with_labels.csv", index=False)


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix


In [19]:
# لو انت بالفعل خزنت البيانات
df = pd.read_csv(r"c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\balanced_logs_with_labels.csv")


In [20]:
# فصل الميزات عن الليبل
X = df.drop(columns=['label'])
y = df['label']

# تحويل الليبل لكود رقمي
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# عمل Scaling للبيانات (مهم جدًا للـ SVM)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

model = SVC(kernel='rbf', C=1, gamma='scale')  # تقدر تجرب linear أو poly حسب الأداء
model.fit(X_train, y_train)


SVC(C=1)

In [23]:
y_pred = model.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


Confusion Matrix:
 [[81  0  0  0]
 [ 0 80  0  0]
 [ 0  0 81  0]
 [ 0  0  0 81]]

Classification Report:
                precision    recall  f1-score   support

          DOS       1.00      1.00      1.00        81
        METER       1.00      1.00      1.00        80
       NORMAL       1.00      1.00      1.00        81
PORT SCANNING       1.00      1.00      1.00        81

     accuracy                           1.00       323
    macro avg       1.00      1.00      1.00       323
 weighted avg       1.00      1.00      1.00       323



In [24]:
import joblib

# حفظ الموديل
joblib.dump(model, r'c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\svm_model.pkl')

# حفظ الـ scaler (مهم عشان نستخدمه مع البيانات الجديدة)
joblib.dump(scaler, r'c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\scaler.pkl')

# حفظ الـ label encoder (عشان نرجع الليبلات الأصلية لما نحتاج)
joblib.dump(le, r'c:\Users\MARIAM\Downloads\project data\RANDOM_3_ATTACKS\RANDOM_3_ATTACKS\label_encoder.pkl')


['c:\\Users\\MARIAM\\Downloads\\project data\\RANDOM_3_ATTACKS\\RANDOM_3_ATTACKS\\label_encoder.pkl']